In [1]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division

In [2]:
def calc_k(k,b):
    return np.sum(b[:,np.newaxis,np.newaxis,np.newaxis,np.newaxis]*k,axis=0)

In [3]:
def heuns(tn,x,function,h):
    k1 = h*function(tn,x)
    k2 = h*function(tn+h,x+k1)
    return (k1+k2)/2

In [4]:
def merson(tn,x,function,h):
    k1 = h*function(tn,x)
    k2 = h*function(tn+h/3,x+k1/3)
    k3 = h*function(tn+h/3,x+(k1+k2)/6)
    k4 = h*function(tn+h/2,x+(k1+k3*3)/8)
    k5 = h*function(tn+h,x+k1/2-3*k3/2+2*k4)
    return k1/6+2*k4/3+k5/6

In [5]:
pos = np.mgrid[0:5,0:5]*10.
pos[0,2,2] = 25.
vel = np.zeros_like(pos)
state = np.array([pos,vel])
import pandas as pd

In [6]:
spring_len = lambda direction,ballz: np.sqrt(np.sum((direction-ballz)**2,axis = 0))
def simple_update(tn,x):
    mass = x[0,:,1:-1,1:-1]
    north = x[0,:,0:-2,1:-1]
    south = x[0,:,2:,1:-1]
    west = x[0,:,1:-1,0:-2]
    east = x[0,:,1:-1,2:]
    
    
    spring_n = spring_len(north,mass)
    spring_s = spring_len(south,mass)
    spring_e = spring_len(east,mass)
    spring_w = spring_len(west,mass)
    
    update_north = (spring_n-L)*(mass-north)/spring_n
    update_south = (spring_s-L)*(mass-south)/spring_s
    update_east = (spring_e-L)*(mass-east)/spring_e
    update_west = (spring_w-L)*(mass-west)/spring_w
    
    result = np.zeros_like(x)
    result[1,:,1:-1,1:-1] = -(k/m)*(update_north + update_south + \
                              update_east + update_west)

    result[0] = x[1]
    return result

In [7]:
k = 30
L = 10
g = 9.81
m = 10

In [184]:
def spring_field_rk( update_function, step = 0.01, filename = None):
    import matplotlib.animation as animation
    global state, frame
    
    frame = 0.

    
    pos = np.mgrid[0:5,0:5]*10.
    vel = np.zeros_like(pos)
    pos[0,2,2] = 25.
    pos[1,2,2] = 25.
    pos[0] += 60
    state = np.array([pos,vel])
    
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-10,np.max(state[0,0])+50)
    plt.xlim(np.min(state[0,1])-20,np.max(state[0,1])+20)

    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    v_springs = []
    for i in range(state.shape[2]):
        v_springs.append(ax.plot([],[], linestyle='--', color='red')[0])
        
    h_springs = []
    for i in range(state.shape[3]):
        h_springs.append(ax.plot([],[], linestyle='--', color = 'red')[0])
    
        
    points, = ax.plot([],[], linestyle='None', marker='o', markersize = 10.)
    
    
    def init():
        
        return h_springs + v_springs + [points, time_text]
    
    def animate(t):
        global state,frame
        
        while frame < 1/30:
            state += merson(tn=t*step,x=state,function= update_function,h=step)
            frame += step
        frame -= 1/30
        state[1,0,:,:] = state[1,0,:,:] * -(state[0,0,:,:] < 0)
        state[0,0,:,:] = np.abs(state[0,0,:,:])

        points.set_data([],[])
        #points.set_data(state[0,1].ravel(),state[0,0].ravel())
        
        time_text.set_text(time_template % (t*step))
        
        for i in range(state.shape[2]):
            v_springs[i].set_data(state[0,1,i,:],state[0,0,i,:])
            
        for i in range(state.shape[3]):
            h_springs[i].set_data(state[0,1,:,i],state[0,0,:,i])
        
        return h_springs + v_springs + [points, time_text]
    
    ani = animation.FuncAnimation(fig, animate, np.arange(10/step),
                              interval=1, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [179]:
spring_field_rk(update_function=simple_update)

In [57]:
c = 0.1

In [129]:
def resistance_update(tn,x):
    mass = x[0,:,1:-1,1:-1]
    north = x[0,:,0:-2,1:-1]
    south = x[0,:,2:,1:-1]
    west = x[0,:,1:-1,0:-2]
    east = x[0,:,1:-1,2:]
    
    
    spring_n = spring_len(north,mass)
    spring_s = spring_len(south,mass)
    spring_e = spring_len(east,mass)
    spring_w = spring_len(west,mass)
    
    update_north = (spring_n-L)*(mass-north)/spring_n
    update_south = (spring_s-L)*(mass-south)/spring_s
    update_east = (spring_e-L)*(mass-east)/spring_e
    update_west = (spring_w-L)*(mass-west)/spring_w
    
    result = np.zeros_like(x)
    result[1,:,1:-1,1:-1] = -(k/m)*(update_north + update_south + \
                              update_east + update_west) - \
                            np.sign(x[1,:,1:-1,1:-1])*c*x[1,:,1:-1,1:-1]**2 

    result[0] = x[1]
    return result

In [130]:
spring_field_rk(update_function=resistance_update)

In [13]:
def resistance_gravity_update(tn,x):
    mass = x[0,:,1:-1,1:-1]
    north = x[0,:,0:-2,1:-1]
    south = x[0,:,2:,1:-1]
    west = x[0,:,1:-1,0:-2]
    east = x[0,:,1:-1,2:]
    
    
    spring_n = spring_len(north,mass)
    spring_s = spring_len(south,mass)
    spring_e = spring_len(east,mass)
    spring_w = spring_len(west,mass)
    
    update_north = (spring_n-L)*(mass-north)/spring_n
    update_south = (spring_s-L)*(mass-south)/spring_s
    update_east = (spring_e-L)*(mass-east)/spring_e
    update_west = (spring_w-L)*(mass-west)/spring_w
    
    result = np.zeros_like(x)
    result[1,:,1:-1,1:-1] = -(k/m)*(update_north + update_south + \
                              update_east + update_west) - \
                            np.sign(x[1,:,1:-1,1:-1])*c*x[1,:,1:-1,1:-1]**2
    
    result[1,0,1:-1,1:-1] -= g

    result[0] = x[1]
    
    return result

In [131]:
spring_field_rk(update_function=resistance_gravity_update)

In [174]:
def galareta_update(tn,x):

    
    north = x[0,:,0:-1,:]
    south = x[0,:,1:,:]
    
    west = x[0,:,:,0:-1]
    east = x[0,:,:,1:]
    
    north_west = x[0,:,0:-1,0:-1]
    south_east = x[0,:,1:,1:]
    north_east = x[0,:,0:-1,1:]
    south_west = x[0,:,1:,0:-1]
    
    spring_v = spring_len(north,south)
    spring_h = spring_len(west,east)
    
    spring_diag = spring_len(north_west,south_east)
    
    spring_diag2 = spring_len(north_east,south_west)
    
    update_north = (spring_v-L)*(south-north)/spring_v
    update_south = (spring_v-L)*(north-south)/spring_v
    update_east = (spring_h-L)*(west-east)/spring_h
    update_west = (spring_h-L)*(east-west)/spring_h
    
    update_south_east = (spring_diag-L*np.sqrt(2))*(north_west-south_east)/spring_diag
    
    update_north_west = (spring_diag-L*np.sqrt(2))*(south_east-north_west)/spring_diag

    update_south_west = (spring_diag2-L*np.sqrt(2))*(north_east-south_west)/spring_diag2
    
    update_north_east = (spring_diag2-L*np.sqrt(2))*(south_west-north_east)/spring_diag2
    
    result = np.zeros_like(x)
    result[1,:,1:,:] += -(k/m)*(update_north)
    result[1,:,0:-1:,:] += -(k/m)*(update_south)
    
    result[1,:,:,1:] += -(k/m)*(update_west)
    result[1,:,:,0:-1] += -(k/m)*(update_east)
    
    result[1,:,1:,1:] += -(k/m)*(update_north_west)
    result[1,:,0:-1,0:-1] += -(k/m)*(update_south_east)
    
    result[1,:,0:-1,1:] += -(k/m)*(update_south_west)
    result[1,:,1:,0:-1] += -(k/m)*(update_north_east)
    
    result[1,:,:,:] -= np.sign(x[1,:,:,:])*c*x[1,:,:,:]**2
    result[1,:,:,:] -= c_*x[1,:,:,:]
    
    result[1,0,:,:] -= g

    result[0] = x[1]
    
    return result

In [268]:
k = 500
L = 13
c = 0.01
c_ = 0.01

In [ ]:
spring_field_rk(update_function=galareta_update,filename="galaretka2")